In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import multioutput

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SzymonGrzebyta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SzymonGrzebyta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SzymonGrzebyta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../disaster.db')
df = pd.read_sql("SELECT * FROM disaster", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


In [3]:
def tokenize(text):
    # lower all text
    text = text.lower()
    
    # Normalize text - removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize -> slit into list
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words_list = stopwords.words('english')
    tokens = [token.strip() for token in tokens if token not in stop_words_list]
    
    # Lemmatization
    lemmed = [WordNetLemmatizer().lemmatize(token, pos = 'v') for token in tokens]
    
    # Steaming
    stemmed = [PorterStemmer().stem(word) for word in lemmed]

    return stemmed

In [4]:
def model():
    #setting pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', multioutput.MultiOutputClassifier (RandomForestClassifier()))
        ])

    # fbeta_score scoring object using make_scorer()
    scorer = 'f1_weighted'

    #model parameters for GridSearchCV
    parameters = {  'vect__max_df': (0.3, 0.5),
                    'clf__estimator__n_estimators': [10, 5],
                    'clf__estimator__min_samples_split': [2, 10]
              }
    cv = GridSearchCV(pipeline, param_grid= parameters, scoring = scorer, verbose =7, cv = 2)

    return cv

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import numpy as np
def eval_metrics(ArrayL, ArrayP, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics    

In [6]:
model = model()
model.fit(X_train, Y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__max_df=0.3;, score=0.532 total time=  36.4s
[CV 2/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__max_df=0.3;, score=0.527 total time=  32.3s
[CV 1/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__max_df=0.5;, score=0.521 total time=  30.8s
[CV 2/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, vect__max_df=0.5;, score=0.529 total time=  30.1s
[CV 1/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, vect__max_df=0.3;, score=0.557 total time=  24.9s
[CV 2/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, vect__max_df=0.3;, score=0.551 total time=  25.3s
[CV 1/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, vect__max_df=0.5;, score=0.546 total time=  24.4s
[CV 2/2] END

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000016FFB1C4AF0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 10],
                         'clf__estimator__n_estimators': [10, 5],
                         'vect__max_df': (0.3, 0.5)},
             scoring='f1_weighted', verbose=7)

In [7]:
Y_pred = model.predict(X_test)

In [10]:
col_names = list(Y.columns.values)
eval_metrics0 = eval_metrics(np.array(Y_test), Y_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.819707   0.841986  0.940731  0.888624
request                 0.893174   0.808850  0.506091  0.622616
offer                   0.995565   0.000000  0.000000  0.000000
aid_related             0.768222   0.728675  0.702929  0.715570
medical_help            0.916699   0.566667  0.114865  0.191011
medical_products        0.950058   0.722222  0.094545  0.167203
search_and_rescue       0.970112   0.666667  0.109756  0.188482
security                0.979946   0.000000  0.000000  0.000000
military                0.967991   0.617647  0.120690  0.201923
water                   0.957192   0.836842  0.454286  0.588889
food                    0.944659   0.830986  0.622144  0.711558
shelter                 0.937717   0.814433  0.355056  0.494523
clothing                0.988430   0.666667  0.181818  0.285714
money                   0.976089   0.750000  0.023810  0.046154
missing_people          0.990551   0.000

C:\Users\SzymonGrzebyta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SzymonGrzebyta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SzymonGrzebyta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SzymonGrzebyta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248

In [23]:
mess =np.array(['I am hungry'])
sol = model.predict(mess)
names = list(Y.columns)
for index,x in enumerate(sol[0]):
    if x == 1:
        print(names[index])



related
request
aid_related
food
direct_report


In [28]:
classification_labels = model.predict(['I am hungry'])[0]

classification_results = dict(zip(df.columns[4:], classification_labels))
print(classification_results)
    
    

{'related': 1, 'request': 1, 'offer': 0, 'aid_related': 1, 'medical_help': 0, 'medical_products': 0, 'search_and_rescue': 0, 'security': 0, 'military': 0, 'water': 0, 'food': 1, 'shelter': 0, 'clothing': 0, 'money': 0, 'missing_people': 0, 'refugees': 0, 'death': 0, 'other_aid': 0, 'infrastructure_related': 0, 'transport': 0, 'buildings': 0, 'electricity': 0, 'tools': 0, 'hospitals': 0, 'shops': 0, 'aid_centers': 0, 'other_infrastructure': 0, 'weather_related': 0, 'floods': 0, 'storm': 0, 'fire': 0, 'earthquake': 0, 'cold': 0, 'other_weather': 0, 'direct_report': 1}


In [22]:
import pickle
pickle.dump(model.best_estimator_, open('disaster_model1.sav', 'wb'))

In [33]:
import joblib
model1 = joblib.load("../disaster_model1.sav")

classification_labels = model1.predict(['I am hungry'])[0]

classification_results = dict(zip(df.columns[4:], classification_labels))
print(classification_results)

{'related': 1, 'request': 1, 'offer': 0, 'aid_related': 1, 'medical_help': 0, 'medical_products': 0, 'search_and_rescue': 0, 'security': 0, 'military': 0, 'water': 0, 'food': 1, 'shelter': 0, 'clothing': 0, 'money': 0, 'missing_people': 0, 'refugees': 0, 'death': 0, 'other_aid': 0, 'infrastructure_related': 0, 'transport': 0, 'buildings': 0, 'electricity': 0, 'tools': 0, 'hospitals': 0, 'shops': 0, 'aid_centers': 0, 'other_infrastructure': 0, 'weather_related': 0, 'floods': 0, 'storm': 0, 'fire': 0, 'earthquake': 0, 'cold': 0, 'other_weather': 0, 'direct_report': 1}
